### Code that plots all the similar name stations with no reservior stations included

In [ ]:
import pandas as pd
import requests
from datetime import datetime
import plotly.graph_objects as go

# === Load the file ===
df_things = pd.read_csv("Things_with_similar_names_no_reservoir.csv")

# === Get unique identifiers ===
unique_identifiers = df_things["identifier"].dropna().unique()

# === Build name lookup for pretty labels ===
df_things["dvrt_station_id"] = df_things["dvrt_station_id"].astype(str)
station_name_lookup = dict(zip(df_things["dvrt_station_id"], df_things["thing_name"]))

# === Loop through each identifier group ===
end_date = datetime.today().strftime("%Y-%m-%d")

for ident in unique_identifiers:
    group_df = df_things[df_things["identifier"] == ident]
    station_ids = group_df["dvrt_station_id"].unique()

    data_dict = {}

    # --- Download data ---
    for station_id in station_ids:
        url = f"https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id={station_id}&end_date={end_date}&f=json"
        try:
            r = requests.get(url)
            if r.status_code == 200:
                json_data = r.json()
                if "data" in json_data:
                    df = pd.DataFrame(json_data["data"], columns=["date", "value"])
                    df["date"] = pd.to_datetime(df["date"])
                    df["value"] = pd.to_numeric(df["value"], errors="coerce")
                    df = df.set_index("date").rename(columns={"value": station_id})
                    data_dict[station_id] = df
        except Exception as e:
            print(f"Error fetching data for station {station_id}: {e}")

    # --- Skip if no data ---
    if not data_dict:
        continue

    # --- Combine into single DataFrame ---
    df_all = pd.concat(data_dict.values(), axis=1)
    df_all = df_all.interpolate(limit_direction="both")

    # --- Plot ---
    fig = go.Figure()
    for station_id in df_all.columns:
        fig.add_trace(go.Scatter(
            x=df_all.index,
            y=df_all[station_id],
            mode="lines",
            name=f"{station_name_lookup.get(station_id, 'Unknown')} ({station_id})",
            connectgaps=False
        ))

    fig.update_layout(
        title=f"Identifier Group: {ident}",
        xaxis_title="Date",
        yaxis_title="Discharge (CFS)",
        template="plotly_white",
        width=1200,
        height=600
    )

    fig.show()